# 1: N 관계 테이블을 ORM으로 구현하기

# 1. DB setting

## db, table 만들기
> app만들기
- python manage.py startapp blog

> settings.py에서 블로그 앱 설치
- 데이터베이스와 연동하려면 만드시 설치


In [ ]:
'''
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'myapp',
    'django_extensions'
    'ajax',
    'blog',
]

'''

In [ ]:
>  urls.py
'''
urlpatterns = [
    path('admin/', admin.site.urls),
    path('ajax/', include('ajax.urls')),
    path('', include('myapp.urls')),
    path('blog/', include('blog.urls')),
]


'''

In [ ]:
> models.py : db 만들기
'''
from django.db import models
from django.utils import timezone

# Create your models here.
class Post(models.Model):
    # 기본키
    # 유저를 지우는 방법
    # 1. # 특정 유저를 지우려면, 모든 것을 지우고 모든 것을 지우거나
    # 2. 유저를 지울 때 글도 같이 지우라는 뜻-> on_delelte= models.CASCADE
    author = models.ForeignKey('auth.User', # 시스템 테이블-> 슈퍼유저 테이블
                            on_delete= models.CASCADE, # 단계별로 진행된다는 뜻
                            )
                                 
    title = models.CharField(max_length = 200)
    text = models.TextField()  # textarea로 받는 부분
    created_data = models.DateTimeField(default =timezone.now)
    published_date = models.DateTimeField(blank = True, # application 관점에서 data가 비어도 되는지
                                            null = True) # 데이터 베이서 관점에서 널값 허용

    # 글을 수정한 날짜
    def publish(self):
        self.published_date = timezone.now()
        self.save()
    
    # 프린트 함수
    def __str__(self):
        return self.title


''

In [ ]:
> admin.py: db 등록하기
    '''
from django.contrib import admin
from blog.models import Post  # root 기준
# Register your models here.

admin.site.register(Post)'''

'''


> DB에 반영하기
- db 이력 추적: python manage.py makemigrations 
- db에 반영:python manamge.py migrate

## 1.2. db에서 post 추가하기

### 1.2.1 브라우저에서 추가
- http://127.0.0.1:8000/admin/blog/post/add/

> url 주소 보면
- post 클릭하면: http://127.0.0.1:8000/admin/blog/post/3/change/
- url이 동적으로 만들어지고 있음.


> Author
- user table이 끼워든거 같은데...

> sqlite browser에서 확인하기
- 테이블: blog_post

In [ ]:
'''
CREATE TABLE "blog_post" (
"id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
"title" varchar(200) NOT NULL, 
"text" text NOT NULL,
"created_data" datetime NOT NULL, 
"published_date" datetime NULL,
"author_id" integer NOT NULL 
REFERENCES "auth_user" ("id") DEFERRABLE INITIALLY DEFERRED)  -> 외래키: auth_user table에서 id로 설정되어 있음.
'''

> 테이블을 살펴보면-> framewoalk 마다 다름
- 장고에서는 id라는 key를 무조건 자동으로 생성함.-> auto increment를 사용
- author_id: author table에 있는 id를 갖고와서 외래키로 잡혀 있음.
- pk는 검색시 많이 사용하기 때문에 문자보다는 숫자를 많이 사용
- user table의 id가 외래키로 사용. post table에서의 기본 키임

## 1.3. coding으로 db 추가하기

> 콘솔에서
- python manage.py shell_plus

- 200218_day6_blog db by shell_plus 파일 참조

# 2. index함수 만들기

In [ ]:
#views.py
'''
from django.shortcuts import render
from django.http import HttpResponse
# Create your views here.


def index(request):
    return HttpResponse('ok')
'''

In [ ]:
# urls.py
'''
from django.urls import path
from . import views 

urlpatterns = [
    path('', views.index),
'''

# 3. 동적 url 다루기

- 정적 url: 고정된 url주소
- 동적 url: 파라미터 -> 동적으로 바뀌면서 매핑되는 파라미터 

In [ ]:
# urls.py
'''
from django.urls import path
from . import views 

urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),
'''

In [ ]:
# views.py
'''
# 동적 url 
# name이라는 파라미터를 usr주소에 사용
# url: blog/abe -> index2함수 호출
def index2(request, name):
    return HttpResponse('ok'+name)


'''

> 실행
- 127.0.0.1:8000/abc

In [ ]:
# urls.py
'''
'''
from django.urls import path
from . import views 

urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),
'''
'''

In [ ]:
# views.py
'''
# 동적 url 
# name이라는 파라미터를 usr주소에 사용
# url: blog/abe -> index2함수 호출
def index2(request, name):
    return HttpResponse('ok'+name)


'''

> 실행
- 127.0.0.1:8000/abc

In [3]:
# urls.py
'''
urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),  # 임의의 문자가 다 올 수 있음.
    path('<int:pk>/detail', views.index3),
    
]'''

"\nurlpatterns = [\n    path('', views.index),\n    # 동적 url 다루기\n    path('<name>/', views.index2),  # 임의의 문자가 다 올 수 있음.\n    path('<int:pk>/detail', views.index3),\n    \n]"

In [4]:
# views.py
'''

def index3(request, pk):
    return HttpResponse('ok' + str(pk))
'''

"\n\ndef index3(request, pk):\n    return HttpResponse('ok' + str(pk))\n"

> 실행
- http://127.0.0.1:8000/blog/123/detail

## 3.1. pk(id)에 해당하는 포스트 기사 읽어보기

In [ ]:
#vies.py
'''
from blog.models import Post


def index3(request, pk):
    # pk: table에서 직접적을 field명을 알 수 없음(실제로는 id임)
    # pk: pk = 1인 값을 찾아야 함.
    p = Post.objects.get(pk = pk)
    return HttpResponse('ok' + p.title)
'''

> 실행
- http://127.0.0.1:8000/blog/3/detail

In [ ]:
# 예외처리: pk 값이 없는 경우-> Page not found errer 발생
'''
def index3(request, pk):
    # pk: table에서 직접적을 field명을 알 수 없음(실제로는 id임)
    # pk: pk = 1인 값을 찾아야 함.
    # p = Post.objects.get(pk = pk)

    # 예외처리
    p = get_object_or_404(Post, pk = pk)
    return HttpResponse('ok' + p.title)

'''

## 3.2. 전체 date 출력

In [ ]:
# urls.py
'''

urlpatterns = [
    path('', views.index),
    # 동적 url 다루기
    path('<name>/', views.index2),  # 임의의 문자가 다 올 수 있음.
    path('<int:pk>/detail', views.index3),
    path('list', views.list)
    
]
'''

In [ ]:
# views.py
'''

def list(request):
    data =Post.objects.all()
    return render (request,'blog/list.html', {'data':data})

'''

In [ ]:
# list.html

'''

My blog<br>


{% for d in data%}
{{d.title}}<br>
{{d.text}}
<hr>

{%endfor%}

'''

> 실행
- 127.0.0.1:8000/blog/list

# 4.  view를 개선하기

> 모든 페이지에 항상 나오는 내용
- 저작권, 주소 등...반복되는 요소들을 매번 html마다 반복하지 않고, 한번에 처리

> base template, sub template
- 템플릿 안에 템플릿을 넣는 개념
- 변경되는 템블릿만 수정
- base template: 저작권, 기본사항 등 -> 한번만 하기
- sub template: 안의 부분만 rendering하는 부분

## 4.1. base templage만들기: base.html

> base template를 사용하려면
- sub template에는 base를 import하고, base 안어 들어갈 부분을 block으로 묶기
- base teplate에서는 sub template를 넣을 부분을 block문으로 알려주기

In [ ]:
# base.html
'''
My blog<br>


<!--  나중에 바뀌는 부분-> 페이지에서 rendering되는 부분 -->
{% block content %}
{% endblock %}


copy right.....<br>
서울 특별시

'''

In [ ]:
# list.html
'''
<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

sub-template<br>

{% for d in data%}
    {{d.title}}<br>
    {{d.text|linebreaks}}  
    <!-- linebreaks: enter -> <br>로 바꿔줌 -->
    <hr>
{%endfor%}

{% endblock %}

'''

## 4.2. detail view만들기: title을 클릭하면 게시물 1개를 보여주기

#### 동적 url로 title 보여주기

In [ ]:
# liist.htmp
'''
<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

sub-template<br>

{% for d in data%}

<!-- get 방식으로 보내려면: a href: /blog/id =... -->
<!-- 동적 url을 가져오려면 -->
    <a href="{{d.pk}}/detail"></a>{{d.title}}</a><br>
    <!-- {{d.text|linebreaks}}    # 목록만 보여주도록 하기 위해서 빼기-->
    <!-- linebreaks: enter -> <br>로 바꿔줌 -->
    <hr>
{%endfor%}

{% endblock %}


'''

In [ ]:
# detail.html
'''

<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

게시물을 보여줘<br>

    {{d.text|linebreaks}}  
    <!-- linebreaks: enter -> <br>로 바꿔줌 -->


{% endblock %}

'''

In [ ]:
# view.py
'''
def list(request):
    data =Post.objects.all()
    return render (request,'blog/list.html', {'data':data})

def detail(request, pk):
    p = get_object_or_404(Post, pk = pk)
    return render(request, 'blog/detail.html', {'d': p})
'''